In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="qwen-qwq-32b")
llm.invoke("what is the capital of india?")

AIMessage(content="\n<think>\nOkay, the user is asking for the capital of India. Let me think. I remember that India's capital isn't Mumbai, even though Mumbai is a major city. Wait, isn't it New Delhi? But sometimes people get confused because Delhi and New Delhi are used interchangeably. Let me make sure. From what I've studied, after India gained independence, they moved the capital from Calcutta to Delhi. But New Delhi is a part of the larger Delhi metropolitan area. So the capital city is New Delhi. But I should double-check to be certain. Let me think of some sources. The government buildings like the Parliament and the President's house are in New Delhi. Yeah, I'm pretty sure that's right. So the answer should be New Delhi.\n</think>\n\nThe capital of India is **New Delhi**. \n\nNew Delhi serves as the administrative and political center of the country, housing key government institutions such as the Parliament of India, the Rashtrapati Bhavan (President's residence), and the Su

In [15]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embeddings.embed_query("hell")[-1]

0.021763162687420845

In [20]:
from langchain_community.embeddings import OllamaEmbeddings
test_doc = "hi embeddings!"
ollama_embeddings = OllamaEmbeddings(model="llama3")
#ollama_embeddings.embed_documents(test_doc)

In [22]:
from langchain_community.vectorstores import FAISS

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)
documents = [ document_1]
db = FAISS.from_documents(documents,ollama_embeddings)
db.add_documents()

In [26]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)


documents = [
    document_2
]
uuids = [str(uuid4()) for _ in range(len(documents))]

db.add_documents(documents=documents, ids=uuids)

['f54a9b1c-6e33-455e-841b-4b8835d85dac']

In [28]:
results = db.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=19674.109375] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [39]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    you are good weather assistent 
        <context>
    {context}
    </context>
    Question : {input}
    """
)

In [33]:
input_text = "what will be the weather tomorrow"
prompt.format(input=input_text)

'Human: \n    you are good weather assistent \n    Question : what will be the weather tomorrow\n    '

In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [41]:
document_chain = create_stuff_documents_chain(llm,prompt=prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    you are good weather assistent \n        <context>\n    {context}\n    </context>\n    Question : {input}\n    '), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10825aa20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1085681d0>, model_name='qwen-qwq-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [42]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x15d552210>, search_kwargs={})

In [43]:
from langchain.chains import create_retrieval_chain
retrieval_chains = create_retrieval_chain(retriever,document_chain)

In [46]:
response = retrieval_chains.invoke({"input":{"what will be weather tomorrow"}})

In [47]:
response['answer']

'\n<think>\nOkay, the user is asking about tomorrow\'s weather. Let me check the context they provided. In the context, the first line says the forecast is cloudy and overcast with a high of 62 degrees. The second part is about their breakfast, which doesn\'t relate to the weather.\n\nSo the question is straightforward. The user wants to know the weather for tomorrow. The answer is right there in the first sentence of the context. I just need to restate that information clearly. I should make sure not to include any extra details unless necessary. The temperature is 62 degrees, and the conditions are cloudy and overcast. \n\nWait, should I mention both cloudy and overcast? The original says "cloudy and overcast," so maybe they\'re emphasizing that it\'s fully overcast. I\'ll include both terms to be accurate. Also, the user might appreciate a friendly tone, so maybe start with "Tomorrow\'s weather will be..." and end with a question mark if it\'s a statement. Let me confirm the questio